In [1]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, callbacks
import tensorflow.keras as keras
from urllib.request import urlopen
import numpy as np
import cv2
from matplotlib import pyplot as plt

from albumentations import (
    HorizontalFlip,
    Transpose, ShiftScaleRotate, Resize, RandomSizedCrop, RandomCrop, Rotate
)

def augment(aug, image):
    image = aug(image=image)['image']
    return image

class CIFAR10AugmentationAndResize(tf.keras.utils.Sequence):
    #Default size is 96 because it is one of the default train image size of mobilenetv2
    def __init__(self, x_set, y_set, batch_size, 
                 augmentations = True,
                 resize = False,
                 height = 32, width = 32,
                 pShiftScaleRotate = .2, pRandomSizedCrop = .2):
        self.resize = resize
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.pShiftScaleRotate = pShiftScaleRotate
        self.pRandomSizedCrop = pRandomSizedCrop      
        self.height = height
        self.width = width
        self.augmentations = augmentations

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        images = [imread(file_name) for file_name in batch_x]
        
        if(self.resize):
            resize = Resize(height=self.height, width=self.width)
        rotate = Rotate()
        #shiftScaleRotate = ShiftScaleRotate(p=self.pShiftScaleRotate)
        randomSizedCrop = RandomSizedCrop(min_max_height=(self.height - self.height * .25 ,self.height), height=self.height, width=self.width, p=self.pRandomSizedCrop)
        horizontalFlip = HorizontalFlip()
        
        for i in range(len(images)):    
            if(self.resize):
                images[i] = augment(resize ,images[i])
            
            if(self.augmentations):
                images[i] = augment(rotate ,images[i])
                images[i] = augment(horizontalFlip ,images[i])
                images[i] = augment(randomSizedCrop ,images[i])
        
        return np.array(images)/255, np.array(batch_y)

In [2]:
def architecture5():
    model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (1, 1), input_shape=(32, 32, 3)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(16, 1,padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(32, (3, 3)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.MaxPooling2D(2,2),
                               tf.keras.layers.ReLU(),
                               tf.keras.layers.Conv2D(64, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(32, 1,padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(64, (3, 3)),
                               tf.keras.layers.BatchNormalization(),   
                               tf.keras.layers.MaxPooling2D(2,2),
                               tf.keras.layers.ReLU(),
                               tf.keras.layers.Conv2D(128, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(64, 1,padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(128, (3, 3)),
                               tf.keras.layers.BatchNormalization(),  
                               tf.keras.layers.MaxPooling2D(2,2),
                               tf.keras.layers.ReLU(),
                               tf.keras.layers.Flatten(),
                               tf.keras.layers.Dense(128, activation='relu'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dropout(.2),
                               tf.keras.layers.Dense(64, activation='relu'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dropout(.2),
                               tf.keras.layers.Dense(10, activation='softmax')
                             ])
    return model
    

In [3]:
class SaveModel(callbacks.Callback):
    def __init__(self, filename):
        super(SaveModel, self).__init__()
        self.bestloss = 99999
        self.filename = filename
        
    def on_epoch_end(self, epoch, logs={}):
        val_loss, val_acc = logs['val_loss'], logs['val_accuracy']
        
        if(epoch % 50 == 0):
            self.model.save("checkpoint{}_{}".format(epoch // 50,self.filename), overwrite=True)
        
        if (self.bestloss > val_loss) :
            self.bestloss = val_loss
            print("\nUpdating best model")
            self.model.save("{}".format(self.filename), overwrite=True)
        return

In [4]:
import os
location =  r"E:\Data\CIFAR10\Original\Train"
files = sorted(os.listdir(location), key=lambda x : int(x[:-5]))
train = [ r"{}\{}".format(location,i) for i in files]

location = r"E:\Data\CIFAR10\Original"
file = "Train_Label.csv"
train_labels = pd.read_csv(r"{}/{}".format(location, file),index_col=0).values

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [6]:
from sklearn.model_selection import KFold
callback = [SaveModel("architecture5.h5")]

batch_size = 32
history = []
kf = KFold(n_splits=5, shuffle=True)

for train_, valid in kf.split(train, train_labels):
    train_x, valid_x = np.array(train)[train_],  np.array(train)[valid]
    train_y, valid_y = np.array(train_labels)[train_],  np.array(train_labels)[valid]
    
    train_seq = CIFAR10AugmentationAndResize(train_x, train_y, batch_size)
    validation_seq = CIFAR10AugmentationAndResize(valid_x, valid_y, batch_size, augmentations=False)
    
    model = architecture5()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    
    
    history.append(model.fit_generator(train_seq, 
                                       validation_data=validation_seq, 
                                       epochs=100, shuffle=True, callbacks=callback))

Epoch 1/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.8299 - accuracy: 0.3479
Updating best model
1250/1250 [==============================] - 35s 28ms/step - loss: 1.8299 - accuracy: 0.3480 - val_loss: 1.5292 - val_accuracy: 0.4528
Epoch 2/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.5533 - accuracy: 0.4450
Updating best model
1250/1250 [==============================] - 28s 22ms/step - loss: 1.5534 - accuracy: 0.4450 - val_loss: 1.3670 - val_accuracy: 0.5161
Epoch 3/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.4556 - accuracy: 0.4832
Updating best model
1250/1250 [==============================] - 28s 22ms/step - loss: 1.4554 - accuracy: 0.4833 - val_loss: 1.1998 - val_accuracy: 0.5780
Epoch 4/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.3904 - accuracy: 0.5106
Updating best model
1250/1250 [==============================] - 28s 22ms/step - loss: 1.3905 - accuracy: 0.5107 - val_loss: 1.1686 - val_a

1250/1250 [==============================] - 27s 21ms/step - loss: 0.9279 - accuracy: 0.6808 - val_loss: 0.7179 - val_accuracy: 0.7482
Epoch 42/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9280 - accuracy: 0.6815 - val_loss: 0.7316 - val_accuracy: 0.7456
Epoch 43/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9296 - accuracy: 0.6828 - val_loss: 0.7085 - val_accuracy: 0.7549
Epoch 44/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9278 - accuracy: 0.6819 - val_loss: 0.7224 - val_accuracy: 0.7524
Epoch 45/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9223 - accuracy: 0.6828 - val_loss: 0.7195 - val_accuracy: 0.7479
Epoch 46/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9135 - accuracy: 0.6864 - val_loss: 0.7204 - val_accuracy: 0.7509
Epoch 47/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9110 - accuracy: 0.6866 - val_loss: 0.7023 

1250/1250 [==============================] - 27s 21ms/step - loss: 0.8227 - accuracy: 0.7195 - val_loss: 0.6917 - val_accuracy: 0.7656
Epoch 91/100
1250/1250 [==============================] - 26s 21ms/step - loss: 0.8279 - accuracy: 0.7164 - val_loss: 0.6612 - val_accuracy: 0.7720
Epoch 92/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8247 - accuracy: 0.7185 - val_loss: 0.6731 - val_accuracy: 0.7702
Epoch 93/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.8184 - accuracy: 0.7175
Updating best model
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8184 - accuracy: 0.7175 - val_loss: 0.6526 - val_accuracy: 0.7767
Epoch 94/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8184 - accuracy: 0.7194 - val_loss: 0.6714 - val_accuracy: 0.7706
Epoch 95/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8200 - accuracy: 0.7194 - val_loss: 0.6664 - val_accuracy: 0.7713
Epoch 96/100
1250/

1250/1250 [==============================] - 28s 23ms/step - loss: 0.9103 - accuracy: 0.6887 - val_loss: 0.7067 - val_accuracy: 0.7538
Epoch 45/100
1250/1250 [==============================] - 29s 23ms/step - loss: 0.9120 - accuracy: 0.6903 - val_loss: 0.7211 - val_accuracy: 0.7506
Epoch 46/100
1250/1250 [==============================] - 28s 23ms/step - loss: 0.9078 - accuracy: 0.6887 - val_loss: 0.6938 - val_accuracy: 0.7620
Epoch 47/100
1250/1250 [==============================] - 28s 23ms/step - loss: 0.9062 - accuracy: 0.6862 - val_loss: 0.7197 - val_accuracy: 0.7524
Epoch 48/100
1250/1250 [==============================] - 28s 23ms/step - loss: 0.8969 - accuracy: 0.6934 - val_loss: 0.6948 - val_accuracy: 0.7587
Epoch 49/100
1250/1250 [==============================] - 28s 23ms/step - loss: 0.8979 - accuracy: 0.6930 - val_loss: 0.7042 - val_accuracy: 0.7546
Epoch 50/100
1250/1250 [==============================] - 28s 23ms/step - loss: 0.9018 - accuracy: 0.6909 - val_loss: 0.6779 

Epoch 99/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8066 - accuracy: 0.7234 - val_loss: 0.6452 - val_accuracy: 0.7802
Epoch 100/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8034 - accuracy: 0.7256 - val_loss: 0.6789 - val_accuracy: 0.7672
Epoch 1/100
1250/1250 [==============================] - 37s 30ms/step - loss: 1.8316 - accuracy: 0.3464 - val_loss: 1.4308 - val_accuracy: 0.4808
Epoch 2/100
1250/1250 [==============================] - 27s 22ms/step - loss: 1.5594 - accuracy: 0.4405 - val_loss: 1.3077 - val_accuracy: 0.5285
Epoch 3/100
1250/1250 [==============================] - 27s 22ms/step - loss: 1.4622 - accuracy: 0.4805 - val_loss: 1.1927 - val_accuracy: 0.5721
Epoch 4/100
1250/1250 [==============================] - 27s 22ms/step - loss: 1.3934 - accuracy: 0.5055 - val_loss: 1.2091 - val_accuracy: 0.5720
Epoch 5/100
1250/1250 [==============================] - 27s 22ms/step - loss: 1.3457 - accuracy: 0.5256 - val_loss

1250/1250 [==============================] - 28s 22ms/step - loss: 0.8953 - accuracy: 0.6928 - val_loss: 0.7393 - val_accuracy: 0.7443
Epoch 55/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.8906 - accuracy: 0.6964 - val_loss: 0.7368 - val_accuracy: 0.7489
Epoch 56/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8950 - accuracy: 0.6913 - val_loss: 0.7158 - val_accuracy: 0.7564
Epoch 57/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8944 - accuracy: 0.6938 - val_loss: 0.7133 - val_accuracy: 0.7529
Epoch 58/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.8891 - accuracy: 0.6944 - val_loss: 0.6984 - val_accuracy: 0.7610
Epoch 59/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8866 - accuracy: 0.6956 - val_loss: 0.6964 - val_accuracy: 0.7637
Epoch 60/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.8784 - accuracy: 0.6988 - val_loss: 0.7199 

KeyboardInterrupt: 

In [7]:
import os
location =  r"E:\Data\CIFAR10\Original\Test"
files = sorted(os.listdir(location), key=lambda x : int(x[:-5]))
test = [ r"{}\{}".format(location,i) for i in files]

location = r"E:\Data\CIFAR10\Original"
file = "Test_Label.csv"
test_labels = pd.read_csv(r"{}/{}".format(location, file),index_col=0).values

from sklearn.model_selection import train_test_split
import random

batch_size = 32
test_seq = CIFAR10AugmentationAndResize(test, test_labels, batch_size,height=96,augmentations=False)

model = architecture5()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.load_weights("architecture5.h5")

model.evaluate_generator(test_seq)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 32, 32, 32)        128       
_________________________________________________________________
batch_normalization_v2_44 (B (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 32, 32, 16)        528       
_________________________________________________________________
batch_normalization_v2_45 (B (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 30, 30, 32)        4640      
_________________________________________________________________
batch_normalization_v2_46 (B (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 15, 32)       

[0.6522712220970434, 0.7781]